In [23]:
%load_ext autoreload
%autoreload 2

from src.model.news_encoder import LookupNewsEncoder, NEConfig
from src.model.user_encoder import (
    NewsSelfAttention,
    PopularityEmbedding,
    PopularityAwareUserEncoder,
    ContentPopularityJointAttention,
    CPJAConfig,
    PEConfig,
    NSAConfig,
    PAUEConfig,
)

from src.data.split import EBNeRDSplit

import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some data for testing


In [2]:
split = EBNeRDSplit()

# News Encoder


In [13]:
news_encoder_config = NEConfig()
lookup_news_encoder = LookupNewsEncoder("bert", news_encoder_config)
print(lookup_news_encoder)
print(lookup_news_encoder.__doc__)
print(lookup_news_encoder.forward.__doc__)

LookupNewsEncoder(
  (fcout): Linear(in_features=768, out_features=400, bias=True)
)


    Songga told us to first use the premade article embedding in one
    of the artifacts as a newencoder.

    So this news encoder lookes up the embeddings of the articles in
    the data artifact and convert them to the desired size, using a
    fully connected layer.

    


        Returns n (the news encodings) for a batch of articles.

        n has shape (batch_size, embedding_size), where embedding_size
        is 400 by default.

        


In [4]:
random_article_id = split.get_random_article_id()
news_embeddings = lookup_news_encoder.forward([random_article_id])
news_embeddings.shape

torch.Size([1, 400])

# User Encoder


In [37]:
n_clicks_in_user_history = 10

### News Self Attention


In [52]:
news_self_attention_config = NSAConfig(n_size=news_encoder_config.get_size_n())
news_self_attention = NewsSelfAttention(news_self_attention_config)
print(news_self_attention)
print(news_self_attention.__doc__)
print(news_self_attention.forward.__doc__)

NewsSelfAttention()


    Implementation of the news self-attention module for
    the popularity-aware user encoder.

    


        Calculates the contextual news representations m, based on the
        news embeddings n outputted by the news encoder.

        n is a tensor of shape (batch_size, N, news_embedding_size)
        m is a tensor of shape (batch_size, N, head_output_size * n_attention_heads)
        where N is the number of clicked articles by the user.

        


In [54]:
n = news_embeddings.unsqueeze(1).repeat(
    1,
    n_clicks_in_user_history,
    1,
)
contextual_news_embeddings = news_self_attention.forward(n)
contextual_news_embeddings.shape

torch.Size([1, 10, 20, 20])
400


torch.Size([1, 10, 400])

### Popularity Embedding


In [21]:
popularity_embedding_config = PEConfig()
popularity_embedding = PopularityEmbedding(popularity_embedding_config)
print(popularity_embedding)
print(popularity_embedding.__doc__)
print(popularity_embedding.forward.__doc__)

PopularityEmbedding()


    Implementation of the popularity embedding module for the popularity-aware user encoder.

    Here, as stated in the paper, news recency and content are removed, to avoid non-
    differentiable quantization operations. So only the click-through rate is used
    in the popularity predictor to calculate the popularity score $s_p$.

    This module does not implement the popularity predictor, this is implemented in
    popularity_predictor.py. But this module does implement the conversion from the
    popularity scores to the popularity embeddings, via a simple linear transformation.

    We discussed with Songga that this is probably the way they implemented it, since
    the paper doesn't specifically discuss how this conversion is done. All it states is:

    "Second, we uniformly quantify the popularity of the i-th clicked news predicted by
    the time-aware news popularity predictor and convert it into an embedding vector $p_i$
    via popularity embeddi

In [29]:
popularity_scores = torch.rand(1, 1)
popularity_embeddings = popularity_embedding.forward(popularity_scores)
popularity_embeddings.shape

torch.Size([1, 1, 100])

### Content Popularity Join Attention


In [30]:
cpja_config = CPJAConfig(
    p_size=popularity_embedding_config.p_size,
    m_size=news_self_attention_config.get_size_m(),
)
cpja = ContentPopularityJointAttention(cpja_config)
print(cpja)
print(cpja.__doc__)
print(cpja.forward.__doc__)

ContentPopularityJointAttention()


    Implementation of the content-popularity joint attention module
    for the popularity-aware user encoder.

    This is based on formula (2) in 3.4 of the paper.

    


        Calculates the user interest embeddings u, based on the
        the popularity embeddings p, and the contextual news
        representations m.

        m is a tensor of shape (batch_size, N, m_size)
        p is a tensor of shape (batch_size, N, p_size)
        u is a tensor of shape (batch_size, m_size)
        where N is the number of clicked articles by the user.

        


In [32]:
user_interests = cpja.forward(contextual_news_embeddings, popularity_embeddings)
user_interests.shape

torch.Size([1, 400])